# Codenation - Data Science
<pre>Autor: Leonardo Simões</pre>

## Desafio 7 - Descubra as melhores notas de matemática do ENEM 2016

Você deverá criar um modelo para prever a nota da prova de matemática de quem participou do ENEM 2016. Para isso, usará Python, Pandas, Sklearn e Regression.

### Detalhes

O contexto do desafio gira em torno dos resultados do ENEM 2016 (disponíveis no arquivo train.csv). Este arquivo, e apenas ele, deve ser utilizado para todos os desafios. Qualquer dúvida a respeito das colunas, consulte o [Dicionário dos Microdados do Enem 2016](https://s3-us-west-1.amazonaws.com/acceleration-assets-highway/data-science/dicionario-de-dados.zip).

No arquivo test.csv crie um modelo para prever nota da prova de matemática (coluna `NU_NOTA_MT`) de quem participou do ENEM 2016. 

Salve sua resposta em um arquivo chamado answer.csv com duas colunas: `NU_INSCRICAO` e `NU_NOTA_MT`.

### Tópicos

Neste desafio você aprenderá:
- Python
- Pandas
- Sklearn
- Regression




## Setup geral

In [108]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

## Análise dos dados

Lendo os arquivos de treino (train) e de teste (test).

In [109]:
df_train = pd.read_csv('train.csv')
df_train.head()

,Unnamed: 0,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,...,Q041,Q042,Q043,Q044,Q045,Q046,Q047,Q048,Q049,Q050
0,1,ed50e8aaa58e7a806c337585efee9ca41f1eb1ad,2016,4314902,Porto Alegre,43,RS,24,M,0.0,...,5.0,A,A,A,A,A,A,A,B,D
1,2,2c3acac4b33ec2b195d77e7c04a2d75727fad723,2016,2304707,Granja,23,CE,17,F,0.0,...,NaN,A,A,C,A,B,A,A,C,A
2,3,f4545f8ccb9ff5c8aad7d32951b3f251a26e6568,2016,2304400,Fortaleza,23,CE,21,F,0.0,...,NaN,A,A,A,A,C,A,A,B,A
3,4,3d6ec248fef899c414e77f82d5c6d2bffbeaf7fe,2016,3304557,Rio de Janeiro,33,RJ,25,F,0.0,...,5.0,C,A,A,A,A,D,A,A,A
4,5,bf896ac8d3ecadd6dba1dfbf50110afcbf5d3268,2016,1302603,Manaus,13,AM,28,M,0.0,...,NaN,A,A,A,A,A,A,A,A,A


In [110]:
df_test = pd.read_csv('test.csv')
df_test.head()

,NU_INSCRICAO,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q006,Q024,Q025,Q026,Q027,Q047
0,73ff9fcc02f0a99919906c942c2e1a1042cdcf98,41,PR,22,F,3,1,1,5,1,...,40.0,420.0,B,A,C,A,A,C,C,A
1,71a95f9f1b91a82c65ad94abbdf9f54e6066f968,21,MA,26,F,3,1,1,8,1,...,100.0,580.0,E,B,C,B,B,B,F,A
2,b38a03232f43b11c9d0788abaf060f7366053b6d,23,CE,21,M,1,1,2,0,2,...,80.0,320.0,E,E,D,B,B,A,NaN,A
3,70b682d9a3636be23f6120fa9d6b164eb3c6002d,15,PA,27,F,3,1,1,8,1,...,NaN,NaN,H,E,G,B,B,A,NaN,A
4,715494628a50142ce8cb17191cfe6d0f3cae0934,41,PR,18,M,1,1,2,0,2,...,0.0,320.0,D,H,H,C,B,A,NaN,A


Antes de manipular os dataframes, deve-separar as colunas de notas da prova de matemática do treino e a do número de inscrição do teste.

In [111]:
train_y = df_train['NU_NOTA_MT'].fillna(0)

In [112]:
n_insc = df_test['NU_INSCRICAO'].values

Idealmente os arquivos de teste e treino teriam as mesmas colunas, exceto a que deve ser predita. 
Então, primeiro verifica-se a quantidade de colunas de cada, e depois exclui-se as colunas que não pertence a ambas.

In [113]:
len(df_test.columns)

47

In [114]:
len(df_train.columns)

167

In [115]:
colunas_intersecao = np.intersect1d(df_train.columns.values, df_test.columns.values)
colunas_intersecao

array(['CO_PROVA_CH', 'CO_PROVA_CN', 'CO_PROVA_LC', 'CO_PROVA_MT',
       'CO_UF_RESIDENCIA', 'IN_BAIXA_VISAO', 'IN_CEGUEIRA',
       'IN_DISCALCULIA', 'IN_DISLEXIA', 'IN_GESTANTE', 'IN_IDOSO',
       'IN_SABATISTA', 'IN_SURDEZ', 'IN_TREINEIRO', 'NU_IDADE',
       'NU_INSCRICAO', 'NU_NOTA_CH', 'NU_NOTA_CN', 'NU_NOTA_COMP1',
       'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5',
       'NU_NOTA_LC', 'NU_NOTA_REDACAO', 'Q001', 'Q002', 'Q006', 'Q024',
       'Q025', 'Q026', 'Q027', 'Q047', 'SG_UF_RESIDENCIA',
       'TP_ANO_CONCLUIU', 'TP_COR_RACA', 'TP_DEPENDENCIA_ADM_ESC',
       'TP_ENSINO', 'TP_ESCOLA', 'TP_LINGUA', 'TP_NACIONALIDADE',
       'TP_PRESENCA_CH', 'TP_PRESENCA_CN', 'TP_PRESENCA_LC', 'TP_SEXO',
       'TP_STATUS_REDACAO', 'TP_ST_CONCLUSAO'], dtype=object)

In [116]:
df_train = df_train[colunas_intersecao]
df_train.head()

,CO_PROVA_CH,CO_PROVA_CN,CO_PROVA_LC,CO_PROVA_MT,CO_UF_RESIDENCIA,IN_BAIXA_VISAO,IN_CEGUEIRA,IN_DISCALCULIA,IN_DISLEXIA,IN_GESTANTE,...,TP_ENSINO,TP_ESCOLA,TP_LINGUA,TP_NACIONALIDADE,TP_PRESENCA_CH,TP_PRESENCA_CN,TP_PRESENCA_LC,TP_SEXO,TP_STATUS_REDACAO,TP_ST_CONCLUSAO
0,9cd70f1b922e02bd33453b3f607f5a644fb9b1b8,16f84b7b3d2aeaff7d2f01297e6b3d0e25c77bb2,01af53cd161a420fff1767129c10de560cc264dd,97caab1e1533dba217deb7ef41490f52e459ab01,43,0,0,0,0,0,...,NaN,1,1,1,1,1,1,M,1.0,1
1,909237ab0d84688e10c0470e2997348aff585273,b9b06ce8c319a3df2158ea3d0aef0f7d3eecaed7,01af53cd161a420fff1767129c10de560cc264dd,97caab1e1533dba217deb7ef41490f52e459ab01,23,0,0,0,0,0,...,1.0,2,1,1,1,1,1,F,1.0,2
2,2d22ac1d42e6187f09ee6c578df187a760123ccf,2d22ac1d42e6187f09ee6c578df187a760123ccf,2d22ac1d42e6187f09ee6c578df187a760123ccf,2d22ac1d42e6187f09ee6c578df187a760123ccf,23,0,0,0,0,0,...,NaN,1,1,1,0,0,0,F,NaN,3
3,2d22ac1d42e6187f09ee6c578df187a760123ccf,2d22ac1d42e6187f09ee6c578df187a760123ccf,2d22ac1d42e6187f09ee6c578df187a760123ccf,2d22ac1d42e6187f09ee6c578df187a760123ccf,33,0,0,0,0,0,...,NaN,1,0,1,0,0,0,F,NaN,1
4,2d22ac1d42e6187f09ee6c578df187a760123ccf,2d22ac1d42e6187f09ee6c578df187a760123ccf,2d22ac1d42e6187f09ee6c578df187a760123ccf,2d22ac1d42e6187f09ee6c578df187a760123ccf,13,0,0,0,0,0,...,NaN,1,1,1,0,0,0,M,NaN,1


In [117]:
df_test = df_test[colunas_intersecao]
df_test.head()

,CO_PROVA_CH,CO_PROVA_CN,CO_PROVA_LC,CO_PROVA_MT,CO_UF_RESIDENCIA,IN_BAIXA_VISAO,IN_CEGUEIRA,IN_DISCALCULIA,IN_DISLEXIA,IN_GESTANTE,...,TP_ENSINO,TP_ESCOLA,TP_LINGUA,TP_NACIONALIDADE,TP_PRESENCA_CH,TP_PRESENCA_CN,TP_PRESENCA_LC,TP_SEXO,TP_STATUS_REDACAO,TP_ST_CONCLUSAO
0,9cd70f1b922e02bd33453b3f607f5a644fb9b1b8,16f84b7b3d2aeaff7d2f01297e6b3d0e25c77bb2,01abbb7f1a90505385f44eec9905f82ca2a42cfd,81d0ee00ef42a7c23eb04496458c03d4c5b9c31a,41,0,0,0,0,0,...,NaN,1,0,1,1,1,1,F,1.0,1
1,f48d390ab6a2428e659c37fb8a9d00afde621889,c8328ebc6f3238e06076c481bc1b82b8301e7a3f,72f80e4b3150c627c7ffc93cfe0fa13a9989b610,577f8968d95046f5eb5cc158608e12fa9ba34c85,21,0,0,0,0,0,...,NaN,1,1,1,1,1,1,F,1.0,1
2,9cd70f1b922e02bd33453b3f607f5a644fb9b1b8,16f84b7b3d2aeaff7d2f01297e6b3d0e25c77bb2,01af53cd161a420fff1767129c10de560cc264dd,97caab1e1533dba217deb7ef41490f52e459ab01,23,0,0,0,0,0,...,3.0,2,0,1,1,1,1,M,1.0,2
3,2d22ac1d42e6187f09ee6c578df187a760123ccf,2d22ac1d42e6187f09ee6c578df187a760123ccf,2d22ac1d42e6187f09ee6c578df187a760123ccf,2d22ac1d42e6187f09ee6c578df187a760123ccf,15,0,0,0,0,0,...,NaN,1,0,1,0,0,0,F,NaN,1
4,942ab3dc020af4cf53740b6b07e9dd7060b24164,66b1dad288e13be0992bae01e81f71eca1c6e8a6,5aebe5cad7fabc1545ac7fba07a4e6177f98483c,767a32545304ed293242d528f54d4edb1369f910,41,0,0,0,0,0,...,1.0,2,1,1,1,1,1,M,1.0,2


In [118]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13730 entries, 0 to 13729
Data columns (total 47 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   CO_PROVA_CH             13730 non-null  object 
 1   CO_PROVA_CN             13730 non-null  object 
 2   CO_PROVA_LC             13730 non-null  object 
 3   CO_PROVA_MT             13730 non-null  object 
 4   CO_UF_RESIDENCIA        13730 non-null  int64  
 5   IN_BAIXA_VISAO          13730 non-null  int64  
 6   IN_CEGUEIRA             13730 non-null  int64  
 7   IN_DISCALCULIA          13730 non-null  int64  
 8   IN_DISLEXIA             13730 non-null  int64  
 9   IN_GESTANTE             13730 non-null  int64  
 10  IN_IDOSO                13730 non-null  int64  
 11  IN_SABATISTA            13730 non-null  int64  
 12  IN_SURDEZ               13730 non-null  int64  
 13  IN_TREINEIRO            13730 non-null  int64  
 14  NU_IDADE                13730 non-null

Em um momento anterior eu usei todas as colunas numéricas na predição, mas isso se mostrou menos eficaz do que usar apenas as colunas de notas.

In [119]:
colunas_numericas = df_train.select_dtypes(include=['float64', 'int64']).columns
colunas_numericas 

Index(['CO_UF_RESIDENCIA', 'IN_BAIXA_VISAO', 'IN_CEGUEIRA', 'IN_DISCALCULIA',
       'IN_DISLEXIA', 'IN_GESTANTE', 'IN_IDOSO', 'IN_SABATISTA', 'IN_SURDEZ',
       'IN_TREINEIRO', 'NU_IDADE', 'NU_NOTA_CH', 'NU_NOTA_CN', 'NU_NOTA_COMP1',
       'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5',
       'NU_NOTA_LC', 'NU_NOTA_REDACAO', 'TP_ANO_CONCLUIU', 'TP_COR_RACA',
       'TP_DEPENDENCIA_ADM_ESC', 'TP_ENSINO', 'TP_ESCOLA', 'TP_LINGUA',
       'TP_NACIONALIDADE', 'TP_PRESENCA_CH', 'TP_PRESENCA_CN',
       'TP_PRESENCA_LC', 'TP_STATUS_REDACAO', 'TP_ST_CONCLUSAO'],
      dtype='object')

In [120]:
colunas_notas = ['NU_NOTA_CH', 'NU_NOTA_CN', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2','NU_NOTA_COMP3',
                 'NU_NOTA_COMP4','NU_NOTA_COMP5','NU_NOTA_LC', 'NU_NOTA_REDACAO']

In [121]:
df_train = df_train[colunas_notas].fillna(0)

In [122]:
df_test = df_test[colunas_notas].fillna(0)

In [123]:
sc = StandardScaler()
x_train = sc.fit_transform(df_train)
x_test = sc.transform(df_test)

In [124]:
lm = LinearRegression()
lm.fit(x_train, train_y)
y_teste = lm.predict(x_test)
y_teste = [round(y,1) for y in y_teste]

Depois de predizer as notas do arquivo de teste, estas foram salvar em um arquivo csv.

In [125]:
answer = pd.DataFrame()
answer['NU_INSCRICAO'] = n_insc
answer['NU_NOTA_MT'] = y_teste
answer.head()

,NU_INSCRICAO,NU_NOTA_MT
0,73ff9fcc02f0a99919906c942c2e1a1042cdcf98,424.4
1,71a95f9f1b91a82c65ad94abbdf9f54e6066f968,482.4
2,b38a03232f43b11c9d0788abaf060f7366053b6d,564.5
3,70b682d9a3636be23f6120fa9d6b164eb3c6002d,-3.0
4,715494628a50142ce8cb17191cfe6d0f3cae0934,547.7


In [126]:
answer.to_csv('answer.csv', index=False)